In [17]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns

import re

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

from ast import literal_eval
import lightgbm as lgbm
import xgboost as xgb

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.feature_extraction.text import TfidfVectorizer

import datetime

from sklearn.ensemble import GradientBoostingRegressor

In [94]:
data = pd.read_csv('movies_metadata.csv')
data = data.drop([19730,29503,35587])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  object 
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

C:\Users\wanga\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [97]:
# Only get data that belongs to a collection
import ast
film = data.copy()
film = film[film['belongs_to_collection'].fillna('no') != 'no'].reset_index(drop = True)
film = film.drop(columns = ['adult', 'homepage','id','imdb_id','tagline','poster_path','original_title'])
film['belongs_to_collection'] = film['belongs_to_collection'].apply(ast.literal_eval)
# film['collection'] = film['belongs_to_collection'].apply(get_collection)
film['belongs_to_collection'] = film['belongs_to_collection'].apply(lambda x: x['name'])
film = film[film['release_date'].fillna('no') != 'no'].reset_index(drop = True)
film['release_year'] = film['release_date'].apply(lambda x: int(x.split('-')[0]))
film['release_month'] = film['release_date'].apply(lambda x: int(x.split('-')[1]))
film['vote_average'].unique()

array([ 7.7,  6.5,  5.7,  6.6,  7.1,  6.1,  6.4,  6. ,  5.4,  6.7,  2.8,
        7. ,  6.9,  4.4,  5.2,  6.8,  5.6,  5.5,  4.9,  7.2,  7.4,  8.1,
        4.6,  7.6,  7.8,  7.3,  6.3,  5. ,  8. ,  4.1,  3.4,  4.5,  6.2,
        4.8,  4.2,  7.9,  5.9,  7.5,  8.2,  5.8,  8.5,  5.3,  8.3,  4. ,
        3.5,  2.7,  4.7,  3.8,  5.1,  3.9,  2.9,  4.3,  3.6,  3.2,  3. ,
        3.3,  2.5,  2. ,  0. ,  8.4,  1.5,  3.7,  3.1,  9. ,  2.1,  2.2,
        2.6,  0.5,  2.3,  1.8,  1.9,  9.3,  8.7,  1.7, 10. ,  8.6,  8.9,
        9.4])

In [93]:
film.iloc[0]

belongs_to_collection                                 Toy Story Collection
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
original_language                                                       en
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
release_date                                                    1995-10-30
revenue                                                        373554033.0
runtime                                                               81.0
spoken_languages                  [{'iso_639_1': 'en', 'name': 'English'}]
status                                                            Released
title                    

In [53]:
film.head()

,belongs_to_collection,budget,genres,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,video,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,False,7.7,5415.0
1,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,False,6.5,92.0
2,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,False,5.7,173.0
3,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",en,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,GoldenEye,False,6.6,1194.0
4,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",en,Balto,An outcast half-wolf risks his life to prevent...,12.140733,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,11348324.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Balto,False,7.1,423.0
